In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob, os, shutil
from skimage.feature import peak_local_max
from scipy.signal import find_peaks, peak_widths
from scipy.signal import savgol_filter
from scipy.integrate import simpson
from numpy import trapz

In [2]:
os.chdir('C:/Users/robin/Programs/Candy_Collect/Grant/Database/Dip_Data/')

In [3]:
df = pd.read_pickle('temp\df_dips.pickle')

## Restructures the 1D array back to the original 2D array, and then trims the perimeter to minimize edge effects

In [4]:
for i in range(len(df)):
    df.dips_11_15[i] = df.dips_11_15[i][5:65, 10:260]
    df.dips_11_1[i] = df.dips_11_1[i][5:65, 10:260]
    df.dips_11_22[i] = df.dips_11_22[i][5:65, 10:260]
    df.dips_8_3[i] = df.dips_8_3[i][5:65, 10:260]
    df.dips_9_6[i] = df.dips_9_6[i][5:65, 10:260]    

## Analysis

In [5]:
def peak(img):
    bw_smooth = savgol_filter(np.mean(img, axis=0), 5, 2)
    peaks, _ = find_peaks(bw_smooth, distance=100)
    return bw_smooth[peaks[0]], peaks[1]-peaks[0], bw_smooth, peaks

In [6]:
# a, b, c, d = peak(df.N4[0])

In [7]:
# plt.plot(c)
# plt.plot([d[1] - (110+35), d[1] - (110+35)], [.35, .6])
# plt.plot([d[1] - (110+45), d[1] - (110+45)], [.35, .6])
# plt.plot([d[1] - [(110/2) + 15], d[1] - [(110/2) + 15]], [.35, .6])
# plt.plot([d[1] - [(110/2) + 5], d[1] - [(110/2) + 5]], [.35, .6])
# plt.plot([d[1] - 120, d[1] - 120], [.35, .6])
# plt.plot([d[1] - 95, d[1] - 95], [.35, .6])
# plt.grid()

In [8]:
# np.average(c[int(d[1] - (110+45)):int(d[1] - (110+35))])

In [9]:
# np.average(c[int(d[1] - [(110/2) + 15]):int(d[1] - [(110/2) + 5])])

In [10]:
# np.amax(c[d[1] - 120:d[1] - 95])/np.average(c[int(d[1] - (110+45)):int(d[1] - (110+35))])

In [11]:
# 1.02 + 3*np.std([1.02, 1.01, 1.01, 1.01])

In [12]:
ex1_SBR, ex2_SBR, ex3_SBR, ex4_SBR, ex5_SBR= [], [], [], [], []

for i in range(len(df)):
    a, b, c, d = peak(df.dips_8_3[i])
    base_l = np.average(c[int(d[1] - (110+45)):int(d[1] - (110+35))])
    base_r = np.average(c[int(d[1] - [(110/2) + 15]):int(d[1] - [(110/2) + 5])])
    SBR_test_line = np.amax(c[d[1] - 120:d[1] - 95])/np.average([base_l, base_r])
    ex1_SBR.append(SBR_test_line)
    
for i in range(len(df)):    
    a, b, c, d = peak(df.dips_9_6[i])
    base_l = np.average(c[int(d[1] - (110+45)):int(d[1] - (110+35))])
    base_r = np.average(c[int(d[1] - [(110/2) + 15]):int(d[1] - [(110/2) + 5])])
    SBR_test_line = np.amax(c[d[1] - 120:d[1] - 95])/np.average([base_l, base_r])
    ex2_SBR.append(SBR_test_line)
    
for i in range(len(df)):    
    a, b, c, d = peak(df.dips_11_1[i])
    base_l = np.average(c[int(d[1] - (110+45)):int(d[1] - (110+35))])
    base_r = np.average(c[int(d[1] - [(110/2) + 15]):int(d[1] - [(110/2) + 5])])
    SBR_test_line = np.amax(c[d[1] - 120:d[1] - 95])/np.average([base_l, base_r])
    ex3_SBR.append(SBR_test_line)
    
for i in range(len(df)):    
    a, b, c, d = peak(df.dips_11_15[i])
    base_l = np.average(c[int(d[1] - (110+45)):int(d[1] - (110+35))])
    base_r = np.average(c[int(d[1] - [(110/2) + 15]):int(d[1] - [(110/2) + 5])])
    SBR_test_line = np.amax(c[d[1] - 120:d[1] - 95])/np.average([base_l, base_r])
    ex4_SBR.append(SBR_test_line)
    
for i in range(len(df)):    
    a, b, c, d = peak(df.dips_11_22[i])
    base_l = np.average(c[int(d[1] - (110+45)):int(d[1] - (110+35))])
    base_r = np.average(c[int(d[1] - [(110/2) + 15]):int(d[1] - [(110/2) + 5])])
    SBR_test_line = np.amax(c[d[1] - 120:d[1] - 95])/np.average([base_l, base_r])
    ex5_SBR.append(SBR_test_line)

In [13]:
positivity_threshold = np.average([ex1_SBR[0], ex2_SBR[0], ex3_SBR[0], ex4_SBR[0], ex5_SBR[0]]) + 3*np.std([ex1_SBR[0], ex2_SBR[0], ex3_SBR[0], ex4_SBR[0], ex5_SBR[0]])
positivity_threshold

1.0316452305136878

## Creating a dataframe of results and saving as a csv file

In [14]:
df_dips = pd.DataFrame()

In [15]:
df_dips['ex1'] = ex1_SBR
df_dips['ex2'] = ex2_SBR
df_dips['ex3'] = ex3_SBR
df_dips['ex4'] = ex4_SBR
df_dips['ex5'] = ex5_SBR

In [16]:
df_dips

,ex1,ex2,ex3,ex4,ex5
0,1.011766,1.020856,1.021494,1.006860,1.013893
1,1.135167,1.242562,1.108572,1.120134,1.271792


In [17]:
df_dips.to_csv('temp\SBR_dips.csv', index=False)

In [18]:
test = pd.read_csv('temp\SBR_dips.csv')

In [19]:
test

,ex1,ex2,ex3,ex4,ex5
0,1.011766,1.020856,1.021494,1.006860,1.013893
1,1.135167,1.242562,1.108572,1.120134,1.271792


In [20]:
np.average(test.iloc[0].tolist()) + 3*np.std(test.iloc[0].tolist())

1.0316452305136883